In [3]:
!pip install streamlit -q
!pip install pyngrok -q


In [15]:
%%writefile app.py

import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np
import random
import time

# =====================================================
# PAGE CONFIG
# =====================================================
st.set_page_config(
    page_title="EV Battery Intelligence Dashboard",
    page_icon="🔋",
    layout="wide"
)

# =====================================================
# DATA LOADING
# =====================================================
@st.cache_data
def load_data():
    df_phase7 = pd.read_csv("phase7_anomaly_output.csv")
    df_phase7["timestamp"] = pd.to_datetime(df_phase7["timestamp"])

    df_phase8 = pd.read_csv("phase8_reuse_decisions.csv")

    df = df_phase7.merge(
        df_phase8[["device_id", "reuse_decision"]],
        on="device_id",
        how="left"
    )

    return df, df_phase8

df, latest_status = load_data()

# =====================================================
# SIDEBAR
# =====================================================
st.sidebar.title("Navigation")

page = st.sidebar.radio(
    "Go to",
    [
        "Fleet Overview",
        "Vehicle Details (Live)",
        "Charging Map",
        "Alerts",
        "Reuse Marketplace",
        "Simulation Controls"
    ]
)

# =====================================================
# 1️. FLEET OVERVIEW (WITH LIVE KPI)
# =====================================================
if page == "Fleet Overview":

    st.title("Fleet Overview")

    # ----------------------------
    # STATIC KPIs
    # ----------------------------
    c1, c2, c3 = st.columns(3)

    c1.metric("Total Devices", latest_status["device_id"].nunique())
    c2.metric("Avg SoH", f"{latest_status['soh_smooth'].mean():.2f}")
    c3.metric("High Risk Devices",
              sum(latest_status["risk_level"] == "High"))

    st.divider()

    # ----------------------------
    # LIVE KPI SIMULATION
    # ----------------------------
    st.subheader(" Live Fleet KPI")

    placeholder = st.empty()

    for _ in range(5):
        live_soh = latest_status["soh_smooth"].mean() + random.uniform(-0.01, 0.01)
        live_rul = latest_status["rul_smooth"].mean() + random.uniform(-3, 3)

        with placeholder.container():
            colA, colB = st.columns(2)
            colA.metric("Live Avg SoH", f"{live_soh:.3f}")
            colB.metric("Live Avg RUL", f"{live_rul:.1f}")

        time.sleep(1)

    st.divider()

    # ----------------------------
    # DISTRIBUTIONS
    # ----------------------------
    colA, colB = st.columns(2)

    with colA:
        st.plotly_chart(
            px.histogram(df, x="soh_smooth", nbins=30,
                         title="SoH Distribution"),
            use_container_width=True
        )

    with colB:
        st.plotly_chart(
            px.histogram(df, x="rul_smooth", nbins=30,
                         title="RUL Distribution"),
            use_container_width=True
        )

# =====================================================
# 2️. VEHICLE DETAILS (LIVE TELEMETRY)
# =====================================================
elif page == "Vehicle Details (Live)":

    st.title("Live Vehicle Telemetry")

    device_id = st.selectbox("Select Device",
                             df["device_id"].unique())

    vdf = df[df["device_id"] == device_id].sort_values("timestamp")
    latest = vdf.iloc[-1]

    col1, col2, col3 = st.columns(3)
    col1.metric("SoH", f"{latest['soh_smooth']:.2f}")
    col2.metric("RUL", int(latest["rul_smooth"]))
    col3.metric("Reuse Decision", latest["reuse_decision"])

    st.info("Live Voltage Simulation (Based on SoH)")

    placeholder = st.empty()
    history = []

    # Simulate voltage using SoH relationship
    base_voltage = 3.7 + (latest["soh_smooth"] * 0.4)

    for _ in range(10):
        simulated_voltage = base_voltage + random.uniform(-0.05, 0.05)
        history.append(simulated_voltage)

        with placeholder.container():
            st.plotly_chart(
                px.line(y=history, title="Live Pack Voltage (Simulated)"),
                use_container_width=True
            )

        time.sleep(1)

# =====================================================
# 3️. CHARGING MAP
# =====================================================
elif page == "Charging Map":

    st.title("Fleet Charging Locations")

    device_locations = {
        "EV_RICK_01": [23.2599, 77.4126],
        "EV_SCOOT_01": [28.6139, 77.2090],
        "EV_TAXI_01": [19.0760, 72.8777],
        "NASA_B0005": [34.0522, -118.2437],
        "NASA_B0043": [29.7604, -95.3698],
        "NASA_B0045": [40.7128, -74.0060],
        "NASA_B0049": [37.7749, -122.4194],
        "NASA_B0054": [47.6062, -122.3321],
    }

    map_data = []

    for device in latest_status["device_id"].unique():
        lat, lon = device_locations.get(device, [0, 0])
        map_data.append({
            "latitude": lat,
            "longitude": lon
        })

    map_df = pd.DataFrame(map_data)

    st.map(map_df)

# =====================================================
# 4️. ALERTS
# =====================================================
elif page == "Alerts":

    st.title(" RUL Anomaly Alerts")

    alerts = df[df["rul_anomaly"] == True]

    st.metric("Active RUL Alerts", len(alerts))

    if len(alerts):
        st.dataframe(alerts.head(100))
    else:
        st.success("No anomalies detected")

# =====================================================
# 5️. REUSE MARKETPLACE
# =====================================================
elif page == "Reuse Marketplace":

    st.title(" Battery Reuse Marketplace")

    category = st.selectbox(
        "Select Reuse Decision",
        latest_status["reuse_decision"].unique()
    )

    market_df = latest_status[
        latest_status["reuse_decision"] == category
    ]

    st.metric("Available Batteries", len(market_df))

    st.dataframe(
        market_df.sort_values("soh_smooth", ascending=False)[
            ["device_id", "soh_smooth",
             "rul_smooth", "reuse_decision"]
        ]
    )

# =====================================================
# 6. SIMULATION CONTROLS
# =====================================================
elif page == "Simulation Controls":

    st.title(" Reuse Decision Simulator")

    soh = st.slider("Simulated SoH", 0.0, 1.0, 0.8)
    rul = st.slider("Simulated RUL", 0, 200, 60)
    risk = st.selectbox("Risk Level", ["Low", "Medium", "High"])

    if risk == "High":
        decision = "Immediate Inspection"
    elif soh >= 0.9 and rul >= 60:
        decision = "Continue EV Operation"
    elif soh >= 0.8:
        decision = "Monitor Closely"
    elif soh >= 0.7:
        decision = "Second Life Candidate"
    else:
        decision = "Recycle / End-of-Life"

    k1, k2 = st.columns(2)
    k1.metric("Predicted Decision", decision)
    k2.metric("Risk Level", risk)


Overwriting app.py


In [16]:
!pkill ngrok
!pkill streamlit

!pip install pyngrok --upgrade -q

from pyngrok import ngrok
import subprocess, time

#PASTE YOUR TOKEN HERE
NGROK_AUTH_TOKEN = "39TaCv1V48HbFwBqjHUHZ1bagyZ_84DZmWSUZxjkZjrgN7HJ4"

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# start streamlit
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"]
)

time.sleep(5)

public_url = ngrok.connect(addr=8501)

print("OPEN THIS LINK:")
print(public_url)

OPEN THIS LINK:
NgrokTunnel: "https://irwin-handsomeish-brayan.ngrok-free.dev" -> "http://localhost:8501"
